In [24]:
import pandas as pd
import numpy as np

train = pd.read_csv("train_cleaned.csv")
test  = pd.read_csv("test_cleaned.csv")

print(train.shape, test.shape)
train.head()


(2736, 180) (20, 179)


,Basic_Demos-Enroll_Season,Basic_Demos-Age,Basic_Demos-Sex,CGAS-Season,CGAS-CGAS_Score,Physical-Season,Physical-BMI,Physical-Height,Physical-Weight,Physical-Waist_Circumference,...,SDS-SDS_Total_T_missing,CGAS-CGAS_Score_missing,PreInt_EduHx-computerinternet_hoursday_missing,act_std_zero_flag,SDS_T_log,light_mod_ratio,CGAS_std,sleep_anomaly,internet_low_SDS_high,sensor_zero_variance
0,Fall,5,0,Winter,51.0,Fall,16.877316,46.0,50.8,26.0,...,1,0,0,0,4.025352,NaN,-1.293661,0,0,0
1,Summer,9,0,missing,65.0,Fall,14.035590,48.0,46.0,22.0,...,0,1,0,0,4.174387,NaN,-0.012476,0,0,0
2,Summer,10,1,Fall,71.0,Fall,16.648696,56.5,75.6,26.0,...,0,0,0,0,4.007333,NaN,0.536603,0,0,0
3,Winter,9,0,Fall,71.0,Summer,18.292347,56.0,81.6,26.0,...,0,0,0,0,3.828641,5.527403,0.536603,0,0,0
4,Spring,13,1,Winter,50.0,Summer,22.279952,59.5,112.2,26.0,...,0,0,0,0,4.043051,11.505588,-1.385174,0,0,0


In [25]:
def feature_engineering(df):
    df = df.copy()
    
    # Avoid division by zero
    eps = 1e-6
    
    # ----------------------------
    # 1. INTERACTIONS WITH AGE
    # ----------------------------
    df['BMI_Age'] = df['Physical-BMI'] * df['Basic_Demos-Age']
    df['InternetHours_Age'] = df['PreInt_EduHx-computerinternet_hoursday'] * df['Basic_Demos-Age']
    
    # ----------------------------
    # 2. INTERNET × PHYSICAL HEALTH
    # ----------------------------
    df['BMI_InternetHours'] = df['Physical-BMI'] * df['PreInt_EduHx-computerinternet_hoursday']
    df['InternetHours_HeartRate'] = (
        df['PreInt_EduHx-computerinternet_hoursday'] / (df['Physical-HeartRate'] + eps)
    )
    
    # ----------------------------
    # 3. BODY COMPOSITION RATIOS
    # ----------------------------
    df['BFP_BMI'] = df['BIA-BIA_Fat'] / (df['BIA-BIA_BMI'] + eps)
    df['Muscle_to_Fat'] = df['BIA-BIA_SMM'] / (df['BIA-BIA_FMI'] + eps)
    df['FFMI_to_Fat'] = df['BIA-BIA_FFMI'] / (df['BIA-BIA_Fat'] + eps)
    
    # ----------------------------
    # 4. HYDRATION FEATURES
    # ----------------------------
    df['Hydration_Status'] = df['BIA-BIA_TBW'] / (df['Physical-Weight'] + eps)
    df['ICW_TBW_Ratio'] = df['BIA-BIA_ICW'] / (df['BIA-BIA_TBW'] + eps)
    df['LST_TBW'] = df['BIA-BIA_LST'] / (df['BIA-BIA_TBW'] + eps)

    # ----------------------------
    # 5. METABOLIC FEATURES
    # ----------------------------
    df['BMR_Weight'] = df['BIA-BIA_BMR'] / (df['Physical-Weight'] + eps)
    df['DEE_Weight'] = df['BIA-BIA_DEE'] / (df['Physical-Weight'] + eps)
    df['FatEnergy_BMR'] = df['BIA-BIA_Fat'] * df['BIA-BIA_BMR']
    df['FatEnergy_DEE'] = df['BIA-BIA_Fat'] * df['BIA-BIA_DEE']

    # ----------------------------
    # 6. CARDIO + MOOD INTERACTION (VERY STRONG FEATURES)
    # ----------------------------
    df['HeartRate_Mood'] = df['Physical-HeartRate'] * df['SDS-SDS_Total_T']
    df['SDS_CGAS_Ratio'] = df['SDS-SDS_Total_T'] / (df['CGAS-CGAS_Score'] + eps)
    df['SDS_PAQ_Interaction'] = df['SDS-SDS_Total_T'] * df['PAQ_C-PAQ_C_Total']

    return df


### Tổng quan các nhóm Feature Engineering
| Nhóm FE                                        | Mô tả                                     | Ý nghĩa khoa học                                           | Lợi ích cho mô hình                            | Cột tạo ra                              |
| ---------------------------------------------- | ----------------------------------------- | ---------------------------------------------------------- | ---------------------------------------------- | --------------------------------------- |
| **1. Loại Season**                             | Bỏ các biến Season (spring, summer…)      | Season không liên quan mạnh đến PIU; tránh “nhiễu mô hình” | Giảm chiều dữ liệu, tránh overfit              | —                                       |
| **2. Interaction BMI × Age**                   | Tương tác giữa thể trạng và tuổi          | BMI ảnh hưởng khác nhau theo tuổi (dậy thì vs. trẻ nhỏ)    | XGBoost/TabNet dễ tìm được quan hệ phi tuyến   | `BMI_Age`                               |
| **3. Interaction Internet × Age**              | Tương tác giờ Internet và tuổi            | Các hành vi Internet thay đổi mạnh theo tuổi               | Giúp mô hình phân biệt đối tượng nguy cơ       | `Internet_Hours_Age`                    |
| **4. BMI × Internet**                          | Tương tác dư cân – hành vi Internet       | Lối sống ít vận động thường đồng thời đi kèm PIU           | Tăng khả năng phát hiện yếu tố kết hợp         | `BMI_Internet_Hours`                    |
| **5. Body Composition Ratios**                 | Các tỷ lệ về mỡ, cơ, nước                 | Phản ánh sức khỏe thật, không chỉ cân nặng                 | Dự đoán tốt vì PIU liên quan tới lối sống tĩnh | `BFP_BMI`, `FFMI_BFP`, `LST_TBW`, …     |
| **6. BMR/DEE × Fat Mass**                      | Mức chuyển hóa cơ bản × khối lượng mỡ     | Trẻ ít vận động → BMR thấp → nguy cơ PIU                   | Tạo tín hiệu rõ ràng cho mô hình               | `BFP_BMR`, `BFP_DEE`                    |
| **7. Ratios liên quan Hydration & Body Water** | Nước trong cơ thể vs. cân nặng            | Phản ánh mức độ vận động và sức khỏe                       | Giúp mô hình hiểu trạng thái thể chất tổng thể | `Hydration_Status`, `ICW_TBW`           |
| **8. Chỉ số tim mạch × hành vi**               | Nhịp tim tương tác với SDS, Age           | Stress/Tâm lý tác động trực tiếp tới PIU                   | Phát hiện PIU mức độ Moderate–Severe           | `HeartRate_Age_Ratio`, `HeartRate_SDS`  |
| **9. Stress & Depression Features (SDS)**      | Tương tác SDS với PAQ, HeartRate          | Trầm cảm – stress là nhân tố lớn của PIU                   | Tăng sức mạnh mô hình                          | `SDS_CGAS_Ratio`, `SDS_PAQ_Interaction` |
| **10. Activity Ratio (Internet / PAQ)**        | So sánh Internet và mức hoạt động thực tế | Mất cân bằng hoạt động là dấu hiệu PIU                     | Rất mạnh trong phân loại SII                   | `Internet_to_Activity_Ratio`            |


### Phân tích chi tiết từng feature
| Feature                        | Giải thích dễ hiểu           | Tại sao logic?                                            | Mô hình hưởng lợi thế nào?                |
| ------------------------------ | ---------------------------- | --------------------------------------------------------- | ----------------------------------------- |
| **BMI_Age**                    | BMI * tuổi                   | BMI cao ở tuổi dậy thì nghiêm trọng hơn BMI cao ở trẻ nhỏ | Mô hình hiểu “mức độ ảnh hưởng theo tuổi” |
| **Internet_Hours_Age**         | Giờ Internet × tuổi          | Tuổi 15 lạm dụng Internet nguy hiểm hơn tuổi 7            | Phân chia nhóm nguy cơ chính xác          |
| **BMI_Internet_Hours**         | BMI × Internet               | Dư cân do ít hoạt động, dùng Internet nhiều               | Dễ học ra pattern “sedentary lifestyle”   |
| **BFP_BMI**                    | Mỡ cơ thể / BMI              | BMI cao → do cơ hay do mỡ?                                | Trẻ “bụng mỡ” vs “bụng cơ” phân loại được |
| **FFMI_BFP**                   | Cơ / Mỡ                      | Thể trạng mạnh → ít rủi ro PIU                            | Mô hình đánh giá sức khỏe thực            |
| **LST_TBW**                    | Cơ nạc / nước                | Dấu hiệu thể lực tốt                                      | Mô hình học về chất lượng sức khỏe        |
| **BFP_BMR**                    | Mỡ × chuyển hóa              | Người ít vận động có BMR thấp                             | Chỉ số “lối sống tĩnh” rõ ràng            |
| **SMM_Height**                 | Cơ xương × chiều cao         | Đánh giá fitness                                          | Phân loại trẻ năng động                   |
| **Muscle_to_Fat**              | Cơ / mỡ                      | Fitness tổng thể                                          | Tín hiệu mạnh: trẻ khoẻ → ít PIU          |
| **Hydration_Status**           | Tổng nước / cân nặng         | Sức khoẻ và vận động                                      | Mô hình hiểu chất lượng cơ thể            |
| **Internet_Hours_SDS**         | Internet × mức trầm cảm      | Trầm cảm cao → dễ lạm dụng Internet                       | Giúp mô hình nhận ra PIU nặng             |
| **Internet_to_Activity_Ratio** | Internet / hoạt động         | “Chỉ dùng Internet, không vận động”                       | Dự báo PIU cực mạnh                       |
| **HeartRate_Age_Ratio**        | Nhịp tim / tuổi              | Nhịp tim bất thường theo tuổi → stress                    | Tăng độ nhạy phân loại SII                |
| **SDS_CGAS_Ratio**             | Stress / chức năng toàn diện | Trẻ khó thích nghi → dễ PIU                               | Liên quan y khoa rất rõ                   |
| **SDS_PAQ_Interaction**        | Stress × hoạt động           | Stress cao + ít hoạt động = PIU                           | Mô hình dễ tìm pattern này                |


### Lợi ích chung của toàn bộ Feature Engineering
| Khía cạnh             | Cách FE giúp                           | Tác động đến mô hình                         |
| --------------------- | -------------------------------------- | -------------------------------------------- |
| **Tâm lý – hành vi**  | Tạo các tương tác Internet × SDS × PAQ | Dự đoán PIU nặng chính xác hơn               |
| **Sức khỏe thể chất** | Tạo tỷ lệ mỡ, cơ, nước                 | Mô hình phân biệt trẻ khoẻ / trẻ ít vận động |
| **Chu kỳ phát triển** | Tương tác theo tuổi                    | XGBoost hiểu “context tuổi”                  |
| **Lối sống**          | Internet / hoạt động                   | Feature mạnh nhất cho SII                    |
| **Cải thiện ML**      | Giảm noise, tăng signal                | Accuracy tăng đáng kể, giảm overfit          |


### Phân nhóm Feature phù hợp từng loại model
| Loại mô hình           | Feature mạnh                      | Giải thích                                   |
| ---------------------- | --------------------------------- | -------------------------------------------- |
| **XGBoost / LightGBM** | Tất cả các interaction + ratio    | Mạnh trong học quan hệ phi tuyến             |
| **TabNet**             | Các feature có ý nghĩa domain     | TabNet thích feature chất lượng hơn số lượng |
| **Linear / Logistic**  | Ratio, normalized features        | Tránh multicollinearity                      |
| **Deep Learning**      | Autoencoder embedding + FE tâm lý | Học tốt các pattern phức tạp                 |


In [26]:
train_fe = feature_engineering(train)
test_fe  = feature_engineering(test)

print(train_fe.shape, test_fe.shape)


(2736, 197) (20, 196)


In [27]:
set(train_fe.columns) - set(test_fe.columns)


{'sii'}

In [28]:
common_cols = list(set(train_fe.columns) & set(test_fe.columns))

train_fe = train_fe[common_cols]
test_fe  = test_fe[common_cols]


In [29]:
train_fe.head(5)

,time_of_day_max,anglez_min,X_25%,Y_std,FGC-FGC_GSND_Zone_missing,FGC-FGC_GSD,FGC-FGC_PU_Zone,Z_count,quarter_mean,FGC-FGC_GSD_Zone_missing,...,BIA-BIA_BMI,Physical-BMI,Basic_Demos-Sex_missing,FatEnergy_BMR,Fitness_Endurance-Time_Mins,BIA-BIA_DEE,SDS_PAQ_Interaction,BIA-BIA_SMM,time_of_day_min,InternetHours_Age
0,8.639500e+13,-89.748867,-0.488205,0.476344,1,20.8,0.0,383544.0,2.021691,1,...,16.8792,16.877316,0,8591.822097,7.0,1492.00,140.250,19.5413,0.000000e+00,15.0
1,8.639500e+13,-89.748867,-0.488205,0.476344,1,20.8,0.0,383544.0,2.021691,1,...,14.0371,14.035590,0,3719.320478,7.0,1498.65,149.760,15.4107,0.000000e+00,0.0
2,8.639500e+13,-89.748867,-0.488205,0.476344,0,14.7,1.0,383544.0,2.021691,0,...,17.8506,16.648696,0,17555.646008,7.0,1862.15,117.180,27.1935,0.000000e+00,20.0
3,8.611000e+13,-89.833092,-0.684180,0.502702,1,20.8,0.0,43330.0,3.000000,1,...,18.2943,18.292347,0,21298.377749,9.0,1923.44,110.295,26.4798,5.500000e+10,0.0
4,8.639500e+13,-88.761833,-0.052803,0.303812,0,17.9,0.0,396396.0,3.000000,0,...,30.1865,22.279952,0,90468.027355,7.0,1996.45,230.160,35.3804,0.000000e+00,0.0


In [30]:
OUT_TRAIN = "train_fed.csv"
OUT_TEST  = "test_fed.csv"

train.to_csv(OUT_TRAIN, index=False)
test.to_csv(OUT_TEST, index=False)

print("Saved:", OUT_TRAIN, "and", OUT_TEST)

Saved: train_fed.csv and test_fed.csv
